In [1]:
import numpy as np
import scipy
import pickle
import random
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Set random seed
RANDOM_SEED = 1234
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [3]:
verbose = True
# Generate test graph instances
for num_edges in [500, 1000, 1500]:
    num_nodes = int(np.floor(num_edges*0.6))
    for instance in range(20):
        print(instance)
        points = np.random.uniform(0,1000,size=(num_nodes,2)) # generate random vertex coordinates on 2d plane
        delaunay = scipy.spatial.Delaunay(points) # compute Delaunay triangulation for the points
        
#         # plot graph
#         plt.triplot(points[:,0], points[:,1], delaunay.simplices.copy())
#         plt.plot(points[:,0], points[:,1], 'o')
#         plt.show()
        
        # create networkx graph from graph
        G = nx.Graph()
        G.add_nodes_from(range(num_nodes)) # add a vertex for each spatial point
        indices, indptr = delaunay.vertex_neighbor_vertices # get indices of neighbors of each vertex
        for k in range(num_nodes):
            neighbor_nodes = indptr[indices[k]:indices[k+1]]
            for n in neighbor_nodes:
                G.add_edge(k,n)
        is_planar, embedding = nx.planarity.check_planarity(G)
        assert is_planar, 'Networkx graph created was not planar'

        # remove edges until we have the target number of edges
        G_num_nodes = len(G.nodes())
        G_num_edges = len(G.edges())
        assert len(G.edges()) > num_edges, 'Initial graph has too few edges'
        while G_num_edges > num_edges:
            i = random.sample(range(G_num_edges), 1)[0]
            e_remove = list(G.edges())[i]
            G.remove_edge(e_remove[0], e_remove[1])
            G_num_edges = len(G.edges())
        assert len(G.edges()) == num_edges, 'Graph does not have the target number of edges.'

        # remove any degree 0 nodes
        degrees = list(G.degree())
        for v in degrees:
            if v[1] == 0:
                G.remove_node(v[0])
        G = nx.convert_node_labels_to_integers(G)
        V = [v for v in G.nodes()]
        E = list(G.edges())
        
        if verbose:
            print('Graph has %d nodes and %d edges in total.'%(len(V),len(E)))
            print('Mean degree: %0.2f'%np.mean([v[1] for v in G.degree()]))
        # check if the graph instance has cycles
        if verbose:
            print('The graph instance has %d cycles.'%len(nx.cycle_basis(G)))
            print('---------------------------------------')
        nx.write_graphml(G,"./synthetic_graphs/planar/graphs/G_n_edges_%d_instance_%d.graphml"%(len(E), instance)) # save graph                

        # generate costs
        cost_E = dict()
        for e in E:
            cost = np.random.rand()
            cost_E[e] = cost
        total_cost = sum([cost_E[i] for i in E])
        G_spanning_tree_edges = nx.minimum_spanning_edges(G, data=False)
        total_mst_cost = sum([cost_E[i] for i in G_spanning_tree_edges])
        print(total_mst_cost)
        cost_data = (cost_E, total_cost, total_mst_cost)
        pickle.dump(cost_data, open("./synthetic_graphs/planar/edge_costs/G_n_edges_%d_instance_%d_cost_data.p"%(len(E), instance), "wb"))
        
        # initialize demand between each source-destination pair
        SD = []
        for s in V:
            for d in V:
                SD.append((s,d))
                SD.append((d,s))
        SD = list(set(SD))
        demand = dict.fromkeys(SD)
        for vpair in SD:
            s = vpair[0]
            d = vpair[1]
            if s != d:
                demand[(s,d)] = np.random.randint(1,200)
                demand[(d,s)] = np.random.randint(1,200)
            else:
                demand[(s,d)] = 0
                demand[(d,s)] = 0
        pickle.dump(demand, open("./synthetic_graphs/planar/demands/G_n_edges_%d_instance_%d_demand_data.p"%(len(E), instance), "wb" ))

        # translate demand to values for each edge in isolation (used by supermodular heuristic)
        values = dict()
        for e in E:
            values[e] = demand[(e[0], e[1])] + demand[(e[1], e[0])]
        pickle.dump(values, open("./synthetic_graphs/planar/edge_values/G_n_edges_%d_instance_%d_edge_values_data.p"%(len(E), instance), "wb" ))

        

0
Graph has 297 nodes and 500 edges in total.
Mean degree: 3.37
The graph instance has 204 cycles.
---------------------------------------
152.02846354182404
1
Graph has 297 nodes and 500 edges in total.
Mean degree: 3.37
The graph instance has 205 cycles.
---------------------------------------
156.43197917537427
2
Graph has 293 nodes and 500 edges in total.
Mean degree: 3.41
The graph instance has 208 cycles.
---------------------------------------
149.73042554277183
3
Graph has 299 nodes and 500 edges in total.
Mean degree: 3.34
The graph instance has 203 cycles.
---------------------------------------
147.76969380191133
4
Graph has 297 nodes and 500 edges in total.
Mean degree: 3.37
The graph instance has 204 cycles.
---------------------------------------
158.4514690769106
5
Graph has 296 nodes and 500 edges in total.
Mean degree: 3.38
The graph instance has 206 cycles.
---------------------------------------
147.3448761029001
6
Graph has 292 nodes and 500 edges in total.
Mean deg

436.9462869575317
12
Graph has 890 nodes and 1500 edges in total.
Mean degree: 3.37
The graph instance has 611 cycles.
---------------------------------------
434.2589707180769
13
Graph has 890 nodes and 1500 edges in total.
Mean degree: 3.37
The graph instance has 615 cycles.
---------------------------------------
419.9791902163148
14
Graph has 892 nodes and 1500 edges in total.
Mean degree: 3.36
The graph instance has 609 cycles.
---------------------------------------
435.55906151819073
15
Graph has 891 nodes and 1500 edges in total.
Mean degree: 3.37
The graph instance has 610 cycles.
---------------------------------------
443.6457200214737
16
Graph has 888 nodes and 1500 edges in total.
Mean degree: 3.38
The graph instance has 614 cycles.
---------------------------------------
421.5679339826015
17
Graph has 893 nodes and 1500 edges in total.
Mean degree: 3.36
The graph instance has 609 cycles.
---------------------------------------
436.6436785375062
18
Graph has 890 nodes and 